In [1]:
import numpy as np

In [2]:
def init(params: dict):
    return np.zeros((params["N"], params["D"]), dtype = int)

def check(data: np.array, params: dict):
    for n in range(params["N"]):
        for d in range(params["D"] - 1):
            if data[n][d] == 4 and data[n][d + 1] != 0: return False
    return True

def check_compute_score(shift_count):
    for d in range(params["D"]):
        for index in range(1, 5):
            if shift_count[d, index] > params["beta"] or shift_count[d, index] < params["alpha"]: return False
    return True

def check_constrain_day(shift_count, d):
    for index in range(1, 5):
        if shift_count[d, index] > params["beta"] or shift_count[d, index] < params["alpha"]: return False
    return True

def objective(data: np.array, params: dict):
    max_night_shifts = -1
    for n in range(params["N"]):
        night_shifts = 0
        for d in range(params["D"]):
            if data[n,d] == 4: night_shifts += 1
        max_night_shifts = max(night_shifts, max_night_shifts)
    return max_night_shifts

In [3]:
BEST = [1e8]
BEST_SCHED = [None]

In [18]:
def get_next_index(n, d, params):
    if n < params["N"] - 1 and d < params["D"] - 1: return n + 1, d
    elif n == params["N"] - 1 and d < params["D"] - 1: return 0, d + 1
    else: return None


def GO(data: np.array, shift_count: np.array, F: list, params: dict, d_index: int):
    #print(data)
    #print()
    if check_compute_score(shift_count):
        score = objective(data, params)
        if score < BEST[0]:
            BEST_SCHED[0] = data.copy()
            BEST[0] = score
            #print(shift_count)
            print(score)
            
    day_ranges = [d_index]
    if d_index < params["D"] - 1: day_ranges.append(d_index + 1)
    
    for d in day_ranges:
        #if check_constrain_day(shift_count, d): continue
        for n in range(params["N"]):
            if d in F[n]: continue
            if data[n, d] != 0: continue
            if d > 0 and data[n, d - 1] == 4:
                day_ranges = [0]
            else: day_ranges = range(1, 5)
            for s in day_ranges:
                if s != 0 and shift_count[d, s] >= int(params["N"] / 4) + 1: continue
                data[n, d] = s
                shift_count[d, s] += 1
                try:
                    GO(data, shift_count, F, params, d)
                except RecursionError:
                    shift_count[d, s] -= 1
                    #print("max recursion depth exceeded, retry another way")
                    return
                shift_count[d, s] -= 1
                data[n, d] = 0
    return

In [22]:
params = {"N": 4, "D": 2, "alpha": 1, "beta": 2}
data = init(params)
shift_count = np.zeros((params["D"], 5))
F = [
    [],
    [3],
    [],
    [],
    [],
    [0]
]

In [23]:
BEST = [1e8]
BEST_SCHED = [None]
GO(data, shift_count, F, params, 0)


KeyboardInterrupt



In [21]:
BEST_SCHED

[array([[1],
        [2],
        [3],
        [4]])]

In [8]:
BEST

[1]

In [10]:
import sys